# Best restauration business location in Sydney

The aim of this workbook is to determine the locations where a restaurant business owner is likely to have the more profit by identifying the areas with a high customer density and with not much competition

## Content

    1. Get restaurants data from Foursquare
    2. Clean dataframe
    3. Get ratings and number of ratings for restaurant
    4. First Analysis
    5. Clustering

In [1]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import requests # library to handle requests

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

## 1. Get restaurants data from Foursquare

### A. Define foursquare credentials

In [2]:
CLIENT_ID = 'DZ0YNEFGBRCVQTLGD1QAVINN2NHPCWKUZGDZNVE2ECUMLJV3' # your Foursquare ID
CLIENT_SECRET = 'G5P5NIUFXSJ30WELZBGMD0DLX5I2W5ED3DUNIP1SNRDAIMQC' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50

### B. Search for all the venue of type restaurant around Syney's centre

#### Define the restaurant owner home location

In [3]:
owner_address = '245 Wilson St, Eveleigh NSW 2015, Australia'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(owner_address)
latitude = location.latitude
longitude = location.longitude

print('Address is at lat:', latitude, 'and long:', longitude)

Address is at lat: -33.8941553 and long: 151.19160210316406


#### Search venues of type restaurant in a radius of 10 km of the owner's house

In [4]:
search_query = 'Restaurant'
radius = 10000

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=DZ0YNEFGBRCVQTLGD1QAVINN2NHPCWKUZGDZNVE2ECUMLJV3&client_secret=G5P5NIUFXSJ30WELZBGMD0DLX5I2W5ED3DUNIP1SNRDAIMQC&ll=-33.8941553,151.19160210316406&v=20180604&query=Restaurant&radius=10000&limit=50'

In [5]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e65747260ba08002831f810'},
 'response': {'venues': [{'id': '4dfab6316284607895314beb',
    'name': 'Forum Restaurant',
    'location': {'address': 'Level 1, F23 Administration Building, Eastern Ave',
     'lat': -33.888943,
     'lng': 151.18971,
     'labeledLatLngs': [{'label': 'display',
       'lat': -33.888943,
       'lng': 151.18971}],
     'distance': 605,
     'postalCode': '2008',
     'cc': 'AU',
     'neighborhood': 'Darlington',
     'city': 'Camperdown',
     'state': 'NSW',
     'country': 'Australia',
     'formattedAddress': ['Level 1, F23 Administration Building, Eastern Ave',
      'Camperdown NSW 2008',
      'Australia']},
    'categories': [{'id': '4bf58dd8d48988d169941735',
      'name': 'Australian Restaurant',
      'pluralName': 'Australian Restaurants',
      'shortName': 'Australian',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/australian_',
       'suffix': '.png'},
      'primary': True}],
   

#### Get relevant part of results

In [6]:
# assign relevant part of JSON to venues
venues = results['response']['venues']
venues

[{'id': '4dfab6316284607895314beb',
  'name': 'Forum Restaurant',
  'location': {'address': 'Level 1, F23 Administration Building, Eastern Ave',
   'lat': -33.888943,
   'lng': 151.18971,
   'labeledLatLngs': [{'label': 'display',
     'lat': -33.888943,
     'lng': 151.18971}],
   'distance': 605,
   'postalCode': '2008',
   'cc': 'AU',
   'neighborhood': 'Darlington',
   'city': 'Camperdown',
   'state': 'NSW',
   'country': 'Australia',
   'formattedAddress': ['Level 1, F23 Administration Building, Eastern Ave',
    'Camperdown NSW 2008',
    'Australia']},
  'categories': [{'id': '4bf58dd8d48988d169941735',
    'name': 'Australian Restaurant',
    'pluralName': 'Australian Restaurants',
    'shortName': 'Australian',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/australian_',
     'suffix': '.png'},
    'primary': True}],
  'referralId': 'v-1583707280',
  'hasPerk': False},
 {'id': '4bd66329637ba593a938f870',
  'name': 'Atlas Cafe/restaurant',
  'location': {'

### C. Change to pandas dataframe

In [7]:
df_venues = pd.json_normalize(venues)
df_venues.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,4dfab6316284607895314beb,Forum Restaurant,"[{'id': '4bf58dd8d48988d169941735', 'name': 'A...",v-1583707280,False,"Level 1, F23 Administration Building, Eastern Ave",-33.888943,151.189710,"[{'label': 'display', 'lat': -33.888943, 'lng'...",605,2008,AU,Darlington,Camperdown,NSW,Australia,"[Level 1, F23 Administration Building, Eastern...",NaN,NaN
1,4bd66329637ba593a938f870,Atlas Cafe/restaurant,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",v-1583707280,False,11 Power Ave,-33.900772,151.198825,"[{'label': 'display', 'lat': -33.900772, 'lng'...",993,2015,AU,NaN,Alexandria,NSW,Australia,"[11 Power Ave (Brennan St), Alexandria NSW 201...",Brennan St,NaN
2,4c74516cd8948cfa0bb964da,The Apprentice Restaurant,"[{'id': '4bf58dd8d48988d169941735', 'name': 'A...",v-1583707280,False,Level 7 Building E Ultimo College,-33.881575,151.200867,"[{'label': 'display', 'lat': -33.8815750491685...",1641,2007,AU,NaN,Ultimo,NSW,Australia,[Level 7 Building E Ultimo College (731-695 Ha...,731-695 Harris St.,NaN
3,5607b8d9498eeeaac965cda7,Hannibal Lebanese Restaurant,"[{'id': '58daa1558bbb0b01f18ec1cd', 'name': 'L...",v-1583707280,False,95-97 Glebe Point Rd,-33.882411,151.190242,"[{'label': 'display', 'lat': -33.8824109589435...",1313,2037,AU,NaN,Glebe,NSW,Australia,"[95-97 Glebe Point Rd, Glebe NSW 2037, Australia]",NaN,NaN
4,4b058766f964a520939022e3,Marigold Restaurant 富麗宮 (Marigold Restaurant),"[{'id': '4bf58dd8d48988d1f5931735', 'name': 'D...",v-1583707280,False,4-5/683-689 George St,-33.879703,151.204949,"[{'label': 'display', 'lat': -33.8797028017485...",2027,2000,AU,NaN,Haymarket,NSW,Australia,"[4-5/683-689 George St (at Hay St), Haymarket ...",at Hay St,NaN


## 2. Clean dataframe

#### Rename headers

In [8]:
# headers new name
head = {'location.lat': 'lat', 'location.lng': 'lng', 'location.distance': 'distance'}
df_venues.rename(columns=head, inplace=True)

#### Get type of food

In [9]:
# extract categories from category column
cat = []
for venue in df_venues['categories']:
    cat.append(venue[0]['name'])

df_venues['categories'] = cat

In [10]:
df_venues.head()

,id,name,categories,referralId,hasPerk,location.address,lat,lng,location.labeledLatLngs,distance,location.postalCode,location.cc,location.neighborhood,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,4dfab6316284607895314beb,Forum Restaurant,Australian Restaurant,v-1583707280,False,"Level 1, F23 Administration Building, Eastern Ave",-33.888943,151.189710,"[{'label': 'display', 'lat': -33.888943, 'lng'...",605,2008,AU,Darlington,Camperdown,NSW,Australia,"[Level 1, F23 Administration Building, Eastern...",NaN,NaN
1,4bd66329637ba593a938f870,Atlas Cafe/restaurant,Italian Restaurant,v-1583707280,False,11 Power Ave,-33.900772,151.198825,"[{'label': 'display', 'lat': -33.900772, 'lng'...",993,2015,AU,NaN,Alexandria,NSW,Australia,"[11 Power Ave (Brennan St), Alexandria NSW 201...",Brennan St,NaN
2,4c74516cd8948cfa0bb964da,The Apprentice Restaurant,Australian Restaurant,v-1583707280,False,Level 7 Building E Ultimo College,-33.881575,151.200867,"[{'label': 'display', 'lat': -33.8815750491685...",1641,2007,AU,NaN,Ultimo,NSW,Australia,[Level 7 Building E Ultimo College (731-695 Ha...,731-695 Harris St.,NaN
3,5607b8d9498eeeaac965cda7,Hannibal Lebanese Restaurant,Lebanese Restaurant,v-1583707280,False,95-97 Glebe Point Rd,-33.882411,151.190242,"[{'label': 'display', 'lat': -33.8824109589435...",1313,2037,AU,NaN,Glebe,NSW,Australia,"[95-97 Glebe Point Rd, Glebe NSW 2037, Australia]",NaN,NaN
4,4b058766f964a520939022e3,Marigold Restaurant 富麗宮 (Marigold Restaurant),Dim Sum Restaurant,v-1583707280,False,4-5/683-689 George St,-33.879703,151.204949,"[{'label': 'display', 'lat': -33.8797028017485...",2027,2000,AU,NaN,Haymarket,NSW,Australia,"[4-5/683-689 George St (at Hay St), Haymarket ...",at Hay St,NaN


#### Delete useless columns

In [11]:
df_venues = df_venues[['id', 'name', 'categories', 'lat', 'lng', 'distance']]

In [12]:
df_venues.head()

,id,name,categories,lat,lng,distance
0,4dfab6316284607895314beb,Forum Restaurant,Australian Restaurant,-33.888943,151.189710,605
1,4bd66329637ba593a938f870,Atlas Cafe/restaurant,Italian Restaurant,-33.900772,151.198825,993
2,4c74516cd8948cfa0bb964da,The Apprentice Restaurant,Australian Restaurant,-33.881575,151.200867,1641
3,5607b8d9498eeeaac965cda7,Hannibal Lebanese Restaurant,Lebanese Restaurant,-33.882411,151.190242,1313
4,4b058766f964a520939022e3,Marigold Restaurant 富麗宮 (Marigold Restaurant),Dim Sum Restaurant,-33.879703,151.204949,2027


In [13]:
venue_id = '4b058766f964a520729022e3'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
response = requests.get(url).json()
response

{'meta': {'code': 200, 'requestId': '5e6574697828ae001ba0d40c'},
 'response': {'venue': {'id': '4b058766f964a520729022e3',
   'name': 'Zilver Restaurant',
   'contact': {'phone': '+61292112232', 'formattedPhone': '+61 2 9211 2232'},
   'location': {'address': '477 Pitt St',
    'crossStreet': 'at Hay St',
    'lat': -33.8801834531476,
    'lng': 151.20648884203734,
    'labeledLatLngs': [{'label': 'display',
      'lat': -33.8801834531476,
      'lng': 151.20648884203734}],
    'postalCode': '2000',
    'cc': 'AU',
    'neighborhood': 'Sydney',
    'city': 'Haymarket',
    'state': 'NSW',
    'country': 'Australia',
    'formattedAddress': ['477 Pitt St (at Hay St)',
     'Haymarket NSW 2000',
     'Australia']},
   'canonicalUrl': 'https://foursquare.com/v/zilver-restaurant/4b058766f964a520729022e3',
   'categories': [{'id': '4bf58dd8d48988d145941735',
     'name': 'Chinese Restaurant',
     'pluralName': 'Chinese Restaurants',
     'shortName': 'Chinese',
     'icon': {'prefix': 'htt

## 3. Get ratings and number of ratings for restaurant

We need to call the API for each venue and extraxt the rating and the number of rating

In [14]:
#initialize rating and number of tips
rating = []
nb_rating = []

for venue_id in df_venues['id']:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    response = requests.get(url).json()
    
    #get the overall rating
    try:
        rating.append(response['response']['venue']['rating'])
    except:
        rating.append(np.nan)
        
    #get the number of rating
    try:
        nb_rating.append(response['response']['venue']['ratingSignals'])
    except:
        nb_rating.append(0)

In [15]:
#add to dataframe
df_venues['rating'] = rating
df_venues['number of rating'] = nb_rating
df_venues.head()

,id,name,categories,lat,lng,distance,rating,number of rating
0,4dfab6316284607895314beb,Forum Restaurant,Australian Restaurant,-33.888943,151.189710,605,NaN,0
1,4bd66329637ba593a938f870,Atlas Cafe/restaurant,Italian Restaurant,-33.900772,151.198825,993,NaN,0
2,4c74516cd8948cfa0bb964da,The Apprentice Restaurant,Australian Restaurant,-33.881575,151.200867,1641,NaN,0
3,5607b8d9498eeeaac965cda7,Hannibal Lebanese Restaurant,Lebanese Restaurant,-33.882411,151.190242,1313,NaN,0
4,4b058766f964a520939022e3,Marigold Restaurant 富麗宮 (Marigold Restaurant),Dim Sum Restaurant,-33.879703,151.204949,2027,7.0,296


#### Replace NA rating by average of rating

Places with no rating are considered average from a customer point of view

In [16]:
av_rating = df_venues['rating'].mean(axis=0)
av_rating

6.905882352941176

In [17]:
df_venues['rating'].fillna(av_rating, inplace=True)
df_venues.head()

,id,name,categories,lat,lng,distance,rating,number of rating
0,4dfab6316284607895314beb,Forum Restaurant,Australian Restaurant,-33.888943,151.189710,605,6.905882,0
1,4bd66329637ba593a938f870,Atlas Cafe/restaurant,Italian Restaurant,-33.900772,151.198825,993,6.905882,0
2,4c74516cd8948cfa0bb964da,The Apprentice Restaurant,Australian Restaurant,-33.881575,151.200867,1641,6.905882,0
3,5607b8d9498eeeaac965cda7,Hannibal Lebanese Restaurant,Lebanese Restaurant,-33.882411,151.190242,1313,6.905882,0
4,4b058766f964a520939022e3,Marigold Restaurant 富麗宮 (Marigold Restaurant),Dim Sum Restaurant,-33.879703,151.204949,2027,7.000000,296


In [18]:
#drop id column
df_venues_final = df_venues.drop(columns='id')

## 4. First Analysis

Let's visualize the places with the more ratings on a map (marker size proportional to the number of ratings)

In [19]:
import folium

In [20]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the owner place

In [21]:
# add popular spots to the map as blue circle markers
for lat, lng, label in zip(df_venues_final.lat, df_venues_final.lng, df_venues_final['number of rating']):
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=(label+1)/10,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

venues_map

It looks like the busiest part are around chinatown, town hall and the cbd

Now, let's look at the type of restaurant

In [22]:
df_venues_final.groupby('categories').count().sort_values('name', ascending=False)

,name,lat,lng,distance,rating,number of rating
categories,,,,,,
Chinese Restaurant,6,6,6,6,6,6
Italian Restaurant,6,6,6,6,6,6
Lebanese Restaurant,5,5,5,5,5,5
Japanese Restaurant,3,3,3,3,3,3
Australian Restaurant,3,3,3,3,3,3
Restaurant,3,3,3,3,3,3
Vietnamese Restaurant,2,2,2,2,2,2
Turkish Restaurant,2,2,2,2,2,2
Dim Sum Restaurant,2,2,2,2,2,2


There's a lot of foreign restaurants

Let's look at the restaurant types on a map

In [23]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the owner place

In [24]:
from matplotlib import cm
import matplotlib

In [25]:
color_n = df_venues_final['categories'].unique().size
list_type = df_venues_final['categories'].unique()

# set color scheme for the categories
x = np.arange(color_n)
ys = [i + x + (i*x)**2 for i in range(color_n)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [matplotlib.colors.rgb2hex(i) for i in colors_array]
rainbow_dic = {list_type[i]: rainbow[i] for i in x}
rainbow_dic

{'Australian Restaurant': '#8000ff',
 'Italian Restaurant': '#6a22fe',
 'Lebanese Restaurant': '#5444fd',
 'Dim Sum Restaurant': '#3e65fa',
 'Turkish Restaurant': '#2884f6',
 'Chinese Restaurant': '#11a0f1',
 'Japanese Restaurant': '#04b9ea',
 'Restaurant': '#1acfe3',
 'Seafood Restaurant': '#32e3da',
 'Vietnamese Restaurant': '#48f1d0',
 'Szechuan Restaurant': '#5efac6',
 'Indonesian Restaurant': '#74feba',
 'Asian Restaurant': '#8bfeae',
 'Pizza Place': '#a0faa1',
 'Xinjiang Restaurant': '#b6f193',
 'Korean Restaurant': '#cce385',
 'American Restaurant': '#e4cf74',
 'Cantonese Restaurant': '#fbb965',
 'Portuguese Restaurant': '#ffa055',
 'Hotel Bar': '#ff8444',
 'Malay Restaurant': '#ff6533',
 'Café': '#ff4422',
 'Breakfast Spot': '#ff2211',
 'Fast Food Restaurant': '#ff0000'}

In [26]:
# add popular spots to the map as blue circle markers
for lat, lng, label in zip(df_venues_final.lat, df_venues_final.lng, df_venues_final['categories']):
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color=rainbow_dic[label],
        fill_color=rainbow_dic[label],
        fill_opacity=0.6
        ).add_to(venues_map)

venues_map

Finally, let's look at the ratings of the restaurants on a map (radius proportional to rating)

In [27]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the owner place

In [28]:
min_rating = df_venues_final['rating'].min()
min_rating

5.4

In [29]:
# add popular spots to the map as blue circle markers
for lat, lng, label in zip(df_venues_final.lat, df_venues_final.lng, df_venues_final['rating']):
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=(label-min_rating+0.1)*3.5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

venues_map

Again, the restaurant with the highest ratings are close to the centre (but the ones with low ratings too)

## 5. Clustering restaurants

The restaurants will be clustered based on the following parameters:
    - coordinates (lat and lng)
    - number of ratings
    - overall rating
The type of the restaurant is not used either as it's not a numerical value. It can be used in a second part to localize similar restauration businesses.
A major limitation here is the number of sample. Foursquare (free version) only allows 50 calls to the API for a restaurant data (allowing to obtain the rating and the number of ratings of a restaurant)

In [30]:
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
import sklearn.utils
from sklearn.preprocessing import StandardScaler

In [68]:
# we will find the optimal  umber of clusters with a for loop
k = 10

# variables
clust_dataset = df_venues_final[['lat', 'lng', 'number of rating', 'rating']]

In [69]:
# normalize data to clusterize
clust_dataset = StandardScaler().fit_transform(clust_dataset)

In [71]:
# finding optimal k
for i in range(1, k):
    kmeans = KMeans(n_clusters=i, random_state=0).fit(clust_dataset)
    distance = kmeans.inertia_
    print('k =', i, 'cost =', distance)

k = 1 cost = 200.0
k = 2 cost = 136.75760858229725
k = 3 cost = 92.12856881062015
k = 4 cost = 73.81044904324533
k = 5 cost = 59.03045349843841
k = 6 cost = 50.676383545900904
k = 7 cost = 45.426143718928586
k = 8 cost = 40.807462287720085
k = 9 cost = 36.78160621594749


We will choose k = 6 as for bigger k the distance between points doesn't decrease much

In [72]:
k = 6

In [73]:
kmeans = KMeans(n_clusters=k, random_state=0).fit(clust_dataset)

In [74]:
kmeans.labels_

array([2, 2, 5, 2, 1, 4, 2, 3, 5, 1, 4, 5, 2, 0, 0, 4, 5, 3, 3, 5, 3, 5,
       2, 0, 5, 0, 2, 5, 5, 4, 5, 5, 1, 5, 5, 4, 2, 5, 5, 5, 3, 3, 2, 0,
       5, 1, 3, 5, 4, 5])

In [75]:
df_venues_final['cluster'] = kmeans.labels_

In [76]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=12) # generate map centred around the owner place

In [77]:
# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [matplotlib.colors.rgb2hex(i) for i in colors_array]

In [78]:
# add popular spots to the map as blue circle markers
for lat, lng, label in zip(df_venues_final.lat, df_venues_final.lng, df_venues_final['cluster']):
    folium.vector_layers.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color=rainbow[label-1],
        fill_color=rainbow[label-1],
        fill_opacity=0.6
        ).add_to(venues_map)

venues_map

In [81]:
df_conclu = df_venues_final.groupby('cluster').mean()
df_conclu

,lat,lng,distance,rating,number of rating
cluster,,,,,
0,-33.878861,151.203826,2138.400000,5.780000,47.400000
1,-33.873704,151.206697,2700.250000,7.700000,232.250000
2,-33.886831,151.189258,1190.777778,6.949020,5.555556
3,-33.876235,151.204631,2461.714286,7.728571,68.142857
4,-33.905023,151.172923,2236.333333,6.500980,25.666667
5,-33.876713,151.205779,2415.473684,6.839319,23.052632


In [83]:
df_conclu['number of restaurant'] = df_venues_final.groupby('cluster').count()['rating']

In [96]:
df_conclu

,lat,lng,distance,rating,number of rating,number of restaurant,color
cluster,,,,,,,
0,-33.878861,151.203826,2138.400000,5.780000,47.400000,5,#8000ff
1,-33.873704,151.206697,2700.250000,7.700000,232.250000,4,#1996f3
2,-33.886831,151.189258,1190.777778,6.949020,5.555556,9,#4df3ce
3,-33.876235,151.204631,2461.714286,7.728571,68.142857,7,#b2f396
4,-33.905023,151.172923,2236.333333,6.500980,25.666667,6,#ff964f
5,-33.876713,151.205779,2415.473684,6.839319,23.052632,19,#ff0000


To conclude, cluster 0 includes resturant with low rating and medium number of rating. It can be a good opportunity to be around these restaurants (potentially high number of client but not much competition). Cluster 1 has high number of ratings and a high overall rating. There would be a high competition close to these restaurant, but potentially a high number of customers. Cluster 2 are the closest from the owner place but there seem to be very low customers around.